<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/Preprocess_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
pip install mne matplotlib

     |████████████████████████████████| 7.5 MB 3.8 MB/s 


In [2]:
import numpy as np
import mne 
import pandas as pd
import os
import pylab as pl
import re
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm import tqdm
import shutil
from collections import Counter

tqdm.pandas()

Uploading files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg

/content/drive/.shortcut-targets-by-id/1UmPRvBaUSHwpdFfssLmk_CxDx3urhuBi/isip_tuh_eeg


In [ ]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg

In [19]:
drive_tuh_eeg_loc = Path('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg')

In [13]:
working_folder = Path('/content/drive/MyDrive/MLTSA22/Final_Project/temp')

Creating a dataframe that will contain the name of all the files

In [6]:
downloaded_files = list(Path(drive_tuh_eeg_loc).glob('*'))
downloaded_files[:3]

[PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t003.tse'),
 PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t004.tse'),
 PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t005.tse')]

In [7]:
# how many files we have in total
len(downloaded_files)

13270

In [8]:
files_raw_df = pd.DataFrame(
  [fpath.name for fpath in downloaded_files], 
  columns = ['filename']
)

In [9]:
files_raw_df

,filename
0,train__02_tcp_le__058__00005804__s002_2008_11_...
1,train__02_tcp_le__058__00005804__s002_2008_11_...
2,train__02_tcp_le__058__00005804__s002_2008_11_...
3,train__02_tcp_le__058__00005804__s002_2008_11_...
4,train__02_tcp_le__060__00006083__s003_2010_10_...
...,...
13265,dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13266,dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13267,dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13268,dev__01_tcp_ar__085__00008512__s002_2012_01_09...


In [10]:
files_raw_df['filename'][4].split('__')

['train',
 '02_tcp_le',
 '060',
 '00006083',
 's003_2010_10_11',
 '00006083_s003_t001.tse']

In [11]:
def split_fname(row):

  split_row_str = row['filename'].split('__')

  fname, file_ext = split_row_str[-1].split('.')

  row['category'] = split_row_str[0]
  row['1_idk'] = split_row_str[1]
  row['2_idk'] = split_row_str[2]
  row['3_idk'] = split_row_str[3]
  row['4_idk'] = split_row_str[4]
  row['file_name'] = fname #fname WITHOUT file extension
  row['file_ext'] = file_ext #file extension (.edf or .tse)

  return row

In [12]:
# not the most efficient way, but it's expressive

files_raw_df = files_raw_df.progress_apply(
  split_fname, 
  axis=1
)

100%|██████████| 13270/13270 [01:06<00:00, 198.96it/s]


In [15]:
# saving result
files_raw_df.to_pickle(
    Path(working_folder).joinpath('files_raw_df.pkl')
)

Creating a dictionary with edf files.

In [17]:
files_raw_df['filename']

0        train__02_tcp_le__058__00005804__s002_2008_11_...
1        train__02_tcp_le__058__00005804__s002_2008_11_...
2        train__02_tcp_le__058__00005804__s002_2008_11_...
3        train__02_tcp_le__058__00005804__s002_2008_11_...
4        train__02_tcp_le__060__00006083__s003_2010_10_...
                               ...                        
13265    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13266    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13267    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13268    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13269    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
Name: filename, Length: 13270, dtype: object

In [20]:
file_path = drive_tuh_eeg_loc.joinpath(files_raw_df['filename'][7332])
file_path

PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__053__00005347__s001_2008_11_24__00005347_s001_t001.edf')

In [21]:
edf_fnames_list = files_raw_df[
  files_raw_df['file_ext'] == 'edf'
]['filename']

raw_files_dict = dict()
for fname in tqdm(edf_fnames_list[:50]):
  data_obj = mne.io.read_raw_edf(
    file_path,
    verbose=False,
  )
  raw_files_dict[fname] = data_obj

100%|██████████| 50/50 [00:01<00:00, 25.79it/s]


In [22]:
## access by filename
raw_files_dict[
  'train__03_tcp_ar_a__072__00007221__s003_2010_11_01__00007221_s003_t004.edf'
]

<RawEDF | train__02_tcp_le__053__00005347__s001_2008_11_24__00005347_s001_t001.edf, 41 x 234000 (936.0 s), ~44 kB, data not loaded>

Filling our dataframe with the data from time series.

In [28]:
#The channels that should overlap across different sessions based on the article CITE 
channels = ['EEG FP1-REF',
              'EEG FP2-REF',
              'EEG F3-REF',
              'EEG F4-REF',
              'EEG C3-REF',
              'EEG C4-REF',
              'EEG P3-REF',
              'EEG P4-REF',
              'EEG O1-REF',
              'EEG O2-REF',
              'EEG F7-REF',
              'EEG F8-REF',
              'EEG T3-REF',
              'EEG T4-REF',
              'EEG T5-REF',
              'EEG T6-REF',
              'EEG FZ-REF',
              'EEG CZ-REF',
              'EEG PZ-REF',
              'EEG A1-REF',
              'EEG A2-REF']

In [29]:
work_data = files_raw_df.copy()

In [30]:
#creating a separate column for each of the channels
for i in range(0,len(channels)):
  for name in channels:
    work_data[name] = 'NaN'

In [38]:
work_data = work_data.rename(columns = {'1_idk':'configuration','2_idk':'index','3_idk':'patient_#','4_idk':'session'})

In [46]:
#adding columns for stop and start of the seizure and seizure type
work_data['start'] = 'NaN'
work_data['stop'] = 'NaN'
work_data['seizure'] = 'NaN'

In [62]:
work_data.columns

Index(['filename', 'category', 'configuration', 'index', 'patient_#',
       'session', 'file_name', 'file_ext', 'EEG FP1-REF', 'EEG FP2-REF',
       'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF',
       'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
       'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF',
       'EEG CZ-REF', 'EEG PZ-REF', 'EEG A1-REF', 'EEG A2-REF', 'start', 'stop',
       'seizure'],
      dtype='object')

From here we will take a subset of the data: only AR configuration and get rid of the tse names.

In [48]:
work_data_AR = work_data[work_data['configuration'] == '01_tcp_ar']

In [61]:
work_data_AR = work_data[work_data['file_ext'] == 'edf']

In [69]:
def preprocessing(file):
  #read the data
  data = mne.io.read_raw_edf(file)
  data = data.copy().load_data()
  
  #notch filter 60 Hz - to get rid of the noise because of the power outlets
  eeg_picks = mne.pick_types(data.info, eeg=True)
  freqs = []
  for i in range(0, int(data.info['sfreq']/2), 60):
    if i != 0:
      freqs.append(i)
  data_notch = data.notch_filter(freqs=np.asarray(freqs), picks=eeg_picks)

  #high pass filter
  data_filter = data_notch.filter(l_freq=0.2, h_freq=None)

  #downsampling
  data_done = data_filter.resample(sfreq=200)

  #channel reduction
  print(data.ch_names)
  kick_out = "yes"
  channels2drop = []
  for j in range(0, len(data.ch_names)):
    for i in range(0, len(channels)):
      if data.ch_names[j] == channels[i]:
        kick_out = "no"
        break
      else:
        kick_out = "yes"
    if kick_out == "yes":
      print(f"kick out {j}")
      channels2drop.append(j)
    else:
      print(f"keep {j}")
    names2drop = []
  for i in range(0,len(channels2drop)):
    names2drop.append(data.ch_names[channels2drop[i]])
  for drop in names2drop:
    data.drop_channels(data.ch_names[data.ch_names.index(drop)])

  return data_done

In [93]:
df_columns = work_data_AR.columns #columns in the final data frame
def in_the_dataframe(filename): #takes a name from the filename column without the extension

  #putting an .edf time series in the df
  timeseries = preprocessing(filename)
  data = timeseries.get_data()
  channel_name = timeseries.ch_names
  for name in channel_name:
    for nick in df_columns: #if the channel names in df and our time series piece are the same
      if name == nick:
        pos = channel_name.index(name)
        work_data_AR.at[[tryout['file_name'] == filename], work_data_AR.name] = data[pos]
  
  #getting a name of .tse file
  file_tse = filename[:-3] + 'tse'

  #getting time stamps and seizure type from .tse file
  labels = pd.read_csv(file_tse, sep=" ", index_col=False, header=0, names = ['start','stop','seizure'])
  work_data_AR.at[[work_data_AR['file_name'] == filename], work_data_AR.start] = labels.loc[0,'start']
  work_data_AR.at[[work_data_AR['file_name'] == filename], work_data_AR.stop] = labels.loc[0,'stop']
  work_data_AR.at[[work_data_AR['file_name'] == filename], work_data_AR.seizure] = labels.loc[0,'seizure']

  return work_data_AR
  

In [89]:
tryout = work_data_AR.head(1)

In [92]:
tryout['filename'][6000]

'train__03_tcp_ar_a__072__00007221__s003_2010_11_01__00007221_s003_t003.edf'